In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from   ipywidgets import interact
import kagglehub
import os

In [6]:
path = kagglehub.dataset_download("drnimishadavis/call-center-performance-data")
files = os.listdir(path)
filename = next(f for f in files if f.endswith(".xlsx"))
ful_path = os.path.join(path, filename)
df = pd.read_excel(ful_path)

# Оставим нужные столбцы и удалим пропуски
df_ewma = df[['Date', 'AvgTalkDuration']].copy()
df_ewma.dropna(inplace=True)

# Преобразуем datetime.time в секунды вручную
def time_to_seconds(t):
    if pd.isnull(t):
        return None
    return t.hour * 3600 + t.minute * 60 + t.second + t.microsecond / 1e6

df_ewma['AvgTalkDuration_sec'] = df_ewma['AvgTalkDuration'].apply(time_to_seconds)

# Средняя длительность разговора по дням
avg_talk_by_day = df_ewma.groupby(df_ewma['Date'].dt.date).agg(
    AvgTalkDuration_mean=('AvgTalkDuration_sec', 'mean')
).reset_index()


In [7]:
L = 3
mu_0 = avg_talk_by_day['AvgTalkDuration_mean'].mean()
sigma = avg_talk_by_day['AvgTalkDuration_mean'].std()

In [8]:

def ewma_chart(lambda_):
    ewma_values, ucl_values, lcl_values = [], [], []
    ewma_prev = mu_0

    for i, row in enumerate(avg_talk_by_day.itertuples(), start=1):
        xi = row.AvgTalkDuration_mean
        ewma = lambda_ * xi + (1 - lambda_) * ewma_prev
        ewma_values.append(ewma)
        ewma_prev = ewma

        std_i = sigma * ((lambda_ / (2 - lambda_)) * (1 - (1 - lambda_)**(2 * i)))**0.5
        ucl = mu_0 + L * std_i
        lcl = mu_0 - L * std_i
        ucl_values.append(ucl)
        lcl_values.append(max(0, lcl))

    plt.figure(figsize=(14, 6))
    plt.plot(avg_talk_by_day['Date'], avg_talk_by_day['AvgTalkDuration_mean'], label='Daily Average', linestyle=':', alpha=0.5)
    plt.plot(avg_talk_by_day['Date'], ewma_values, color='blue', marker='o', label='EWMA')
    plt.plot(avg_talk_by_day['Date'], [mu_0] * len(avg_talk_by_day), color='green', linestyle='--', label='CL (Mean)')
    plt.plot(avg_talk_by_day['Date'], ucl_values, color='red', linestyle='--', label='UCL')
    plt.plot(avg_talk_by_day['Date'], lcl_values, color='red', linestyle='--', label='LCL')
    plt.title(f'EWMA Chart: Average Talk Duration (λ = {lambda_:.2f})')
    plt.xlabel('Date')
    plt.ylabel('Average Talk Duration (sec)')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

interact(ewma_chart, lambda_=(0.01, 1.0, 0.01))


interactive(children=(FloatSlider(value=0.5, description='lambda_', max=1.0, min=0.01, step=0.01), Output()), …

<function __main__.ewma_chart(lambda_)>